# Imports 

In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

In [2]:
DB_FILE = "rta_snapshot.db"
engine = sqlite3.connect(DB_FILE)

# Get Data

In [3]:
df = pd.read_sql("select * from battle_logs", engine)

# Data Format

In [4]:
# number of records
df.shape

(58958, 25)

In [5]:
# unique leages
df["P1_LEAGUE"].unique().tolist()

['gold', 'master', 'silver', 'champion', 'challenger', 'bronze', 'legend']

In [6]:
# add win rate + total matches columns

cols = ["P1_WINS", "P1_LOSSES", "P2_WINS", "P2_LOSSES"]
for col in cols:
    df[col] = df[col].astype(int)

In [7]:
# Add some additional columns that might be useful

df["P1_MATCHES"] = df["P1_WINS"] + df["P1_LOSSES"]
df["P2_MATCHES"] = df["P2_WINS"] + df["P2_LOSSES"]

df["P1_WINRATE"] = df["P1_WINS"] / df["P1_MATCHES"]
df["P2_WINRATE"] = df["P2_WINS"] / df["P2_MATCHES"]

df["WINRATE_DELTA"] = df["P1_WINRATE"] - df["P2_WINRATE"]
df["WINRATE_DELTA"] = df["WINRATE_DELTA"].apply(abs)

df["CREATED"] = df["CREATED"].astype(int)
df["TIME"] = df["CREATED"].apply(datetime.fromtimestamp)

# Explore Data

### Get Data by League

In [8]:
# filter by league
legend_p1 = df[df["P1_LEAGUE"] == "legend"]
legend_p2 = df[df["P2_LEAGUE"] == "legend"]

In [9]:
legend_p1.head()

,CREATED,WIN_LOSE,SCORE,P1_SERVER,P2_SERVER,P1_PREBAN,P2_PREBAN,P1_POSTBAN,P2_POSTBAN,P1_PICK1,...,P1_WINS,P1_LOSSES,P2_WINS,P2_LOSSES,P1_MATCHES,P2_MATCHES,P1_WINRATE,P2_WINRATE,WINRATE_DELTA,TIME
223,1612661691,1,3030,world_global,world_global,Fallen Cecilia,Ambitious Tywin,Ravi,Roana,Last Rider Krau,...,320,204,412,312,524,724,0.610687,0.569061,0.041626,2021-02-06 20:34:51
224,1612661398,-1,3020,world_global,world_global,Fallen Cecilia,Fairytale Tenebria,Sage Baal & Sezan,Specter Tenebria,Krau,...,319,204,592,345,523,937,0.609943,0.631804,0.021861,2021-02-06 20:29:58
225,1612653107,1,3029,world_global,world_global,Fallen Cecilia,Fallen Cecilia,Faithless Lidica,Watcher Schuri,Last Rider Krau,...,319,203,1308,1208,522,2516,0.611111,0.519873,0.091238,2021-02-06 18:11:47
226,1612652690,1,3018,world_global,world_global,Fallen Cecilia,Kluri,Remnant Violet,Basar,Krau,...,318,203,237,188,521,425,0.610365,0.557647,0.052718,2021-02-06 18:04:50
264,1612722761,1,3127,world_global,world_global,Kluri,Fallen Cecilia,Ravi,Landy,Fairytale Tenebria,...,447,201,291,354,648,645,0.689815,0.451163,0.238652,2021-02-07 13:32:41


### Look at Preban By League

In [10]:
def get_preban_count(df, key):
    res= df.groupby(key).count()[["CREATED"]].sort_values(by="CREATED").rename(columns={"CREATED" : key.lower()+"_count", "key": "unit"})
    return res

def get_preban_df(p1_df, p2_df):
    prebans = pd.concat([get_preban_count(p1_df, "P1_PREBAN"), get_preban_count(p2_df, "P2_PREBAN")], axis=1).fillna(0)
    prebans["total"] = prebans["p1_preban_count"] + prebans["p2_preban_count"]
    prebans.sort_values("total", ascending=False)
    return prebans

In [11]:
res = {}
# set manually to preserve order
leagues = ["bronze", "silver", "gold", "master", "challenger", "champion", "legend"]

In [12]:
for l in leagues:
    p1_df = df[df["P1_LEAGUE"] == l]
    p2_df = df[df["P2_LEAGUE"] == l]
    final = get_preban_df(p1_df, p2_df)
    final.rename(columns={"total": f"{l}_total_count"}, inplace=True)
    res[l] = final

In [13]:
preban_df = pd.concat([res[k].drop(["p1_preban_count", "p2_preban_count"], axis=1) for k in res.keys()], axis=1)

In [14]:
preban_df = preban_df.fillna(0)

In [15]:
preban_df["total"] = preban_df.sum(axis=1)

In [16]:
preban_df = preban_df.sort_values("total", ascending=False)

In [17]:
preban_df.head()

,bronze_total_count,silver_total_count,gold_total_count,master_total_count,challenger_total_count,champion_total_count,legend_total_count,total
Arbiter Vildred,2853.0,3557.0,4342.0,2315.0,1383.0,1609.0,50.0,16109.0
Cerise,845.0,2011.0,4277.0,3289.0,2364.0,2565.0,37.0,15388.0
Fairytale Tenebria,817.0,1871.0,3999.0,2915.0,2141.0,3345.0,132.0,15220.0
Fallen Cecilia,1427.0,2037.0,2923.0,1901.0,1406.0,2730.0,212.0,12636.0
Last Rider Krau,490.0,978.0,2435.0,2043.0,1994.0,2969.0,133.0,11042.0


In [18]:
# validation
preban_df["total"].sum() / 2 == df.shape[0]

True

In [19]:
preban_df.to_csv("preban_by_league.csv")